# ONNX to TensorRT and inference with TensorRT

This notebook demonstrates onnx format with tensoRT. \
**This notebook is designed for running in Jupyter notebook environment, not for running from python interactive command line. If you would like to execute in python interactive command line, please wrap up all code into a main function (except for import).**

## Install TensorRT
This notebook is created and tested under Ubuntu 20.10, with tools installed of version Ubuntu 20.04 version since 20.10
is not officially supported.

Windows OS is *not* tested

**Must Do**: Make sure you have cuDNN (matches your cuda version) and cuda installed (check nvcc --version)

### Method 1: Native Install
Based on the [installation guide](https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html#installing-pip), we can install tensorRT using pip
alone. 
1. (If you are working on conda virtual environment), follow steps in 
    [this page](https://github.com/stan-dev/pystan/issues/294).
1. Install pycuda using pip, version < 2021.1 
1. From `requirements.txt`. The following line should be added to the `requirements.txt` if we install it from the file
    `--extra-index-url https://pypi.ngc.nvidia.com`.
1. From commandline pip:
    ```bash
    python3 -m pip install --upgrade setuptools pip
    python3 -m pip install nvidia-pyindex
    python3 -m pip install --upgrade nvidia-tensorrt
    ```

### Method 2 (Recommended): Use [TensorRT container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tensorrt)
1. Install Nvidia docker tool chain, following the guide [on this page](https://github.com/NVIDIA/libnvidia-container)
1. You may run into Nvidia docker problems, check if this [link](https://github.com/NVIDIA/nvidia-docker/issues/1447) helps or this [link](https://bbs.archlinux.org/viewtopic.php?id=266915)

## Convert ONNX to tensorrt engine

### Method 1: Using `polygraphy` or `trexec`
This requires trexec being compiled and installed on the system (check [TensorRT git repo](https://github.com/NVIDIA/TensorRT/tree/main/samples/trtexec) for installation guide), 
or if you are using TensorRT container, it is included.

```bash
polygraphy convert dynamic_identity.onnx -o dynamic_identity.engine \
    --trt-min-shapes X:[1,3,28,28] --trt-opt-shapes X:[1,3,28,28] --trt-max-shapes X:[1,3,28,28] \
```
After converting, you can inspect the converted engine with polygraphy to make sure the converted engine is correct.

### Method 2: Using tensorRT libraries (see utility file `src/trt_utils.py`)

In [ ]:
import sys
sys.path.append("../")
from trt_utils import TRTUtils

import cv2
from matplotlib import pyplot as plt
import numpy as np
from model_common import build_pre_process, build_post_process
from model_common.utils import viz_pddle_distill_db_results
# , fourPointsTransform, sorted_boxes, get_image_width_by_height

## Inference using the TensoRT Engine

This part is the same as the normal onnx inference process, but with tensorRT inference
instead of onnxruntime

### Text Detection

Perform preprocessing steps

In [ ]:
test_img_path = "../sample_images/nutrition_label.jpg"
test_img = cv2.imread(test_img_path)
pre_process_op = build_pre_process({"name": "td_PaddlePaddlePreProcess"})
test_in, to_postprocess = pre_process_op(raw_img=test_img, dimension=(960,960))

Run inference with tensorRT

Important to use pycuda here. The autoinit will automatically initialize GPU device.
If not autoinit, an error will occur

In [ ]:
dynamic_trt_config = {
            "fp16":True,
            "original_model_path":  "../models/ch_PP-OCRv2_det_infer.onnx",
            "engine_path":"../models/ch_PP-OCRv2_det_infer.engine",
            "dynamic_shape": True,
            "profile_config":[
                {"x" : [(1, 3, 960, 960), (1, 3, 1280, 1280), (1, 3, 1536,1536) ] }
            ]
        }
static_trt_config = {
    "fp16":True,
    "original_model_path": "../models/ch_PP-OCRv2_det_infer.onnx",
    "engine_path":"../models/ch_PP-OCRv2_det_infer_static.engine",
    "dynamic_shape": False,
    "profile_config":[
        {"x" : [(1, 3, 1088, 1440)] }
    ]
}
trt_utils = TRTUtils(dynamic_trt_config)
trt_utils.get_engine()
output_buffer = trt_utils.inference_single(test_in, profiling=False)

In [ ]:
# reshape
output_shape = list(test_in.shape)
output_shape[1] = 1
output_shape = [1] + output_shape
results = np.reshape(output_buffer[0],output_shape)

Perform post-processing steps

In [ ]:
postprocess_params = {
    "name": "td_PaddlePaddlePostProcess",
    "thresh": 0.3,
    "box_thresh": 0.5,
    "max_candidate": 1000,
    "unclip_ratio": 2,
    "use_dilation": False,
    "score_mode": "fast",
}
post_process_op = build_post_process(postprocess_params)
dt_boxes = post_process_op(results=results, **to_postprocess)

Viz the prediction output

In [ ]:
viz_img = viz_pddle_distill_db_results(test_img.copy(), dt_boxes=dt_boxes)
plt.imshow(cv2.cvtColor(viz_img, cv2.COLOR_BGR2RGB))

In [ ]:
plt.clf()
plt.close()

### Text Recognition